# Lade Daten per Twitter API
*Es werden Timeline Daten für jeden aktuellen Bundestags Politiker geladen, der auf Twitter aktiv ist. 
Die Daten werden anschließend in MongoDB abgelegt, wobei jeder Tweet einem Dokument entspricht. Die Namen der User wurden von den Fraktionsseiten gecrawled.*

### API Verbindung aufsetzen

In [1]:
# Tweepy statt twitter Paket
# https://www.earthdatascience.org/courses/earth-analytics-python/using-apis-natural-language-processing-twitter/get-and-use-twitter-data-in-python/

import os
import tweepy as tw
import pandas as pd

# Die folgenden Zugangsaten können bei Twitter Developer abgerufen werden, sobald ein Account erstellt wurde
consumer_key = os.getenv('consumer_key')
consumer_secret = os.getenv('consumer_secret')
access_token_key = os.getenv('access_token_key')
access_token_secret = os.getenv('access_token_secret')

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token_key, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

### API Verbindung und speichern in Mongo DB
In der Collection Twitter_mdp wird jeder Tweet mit allen Metainformationen als ein Dokument gespeichert. Es werden Tweets zu allen auf Twitter aktiven Politikern einer Fraktion gesammelt. Die entsprechenden Namen wurden von den Bundestagsfraktionsseiten gecrawled.

Anfang Juni 2020 wurde zur Qualitätsicherung des Datensatzes außerdem eine manuelle Kontrolle durchgeführt. Dabei fiel auf, dass einige Abgeordnete einen Twitter Account besitzen, dieser aber nicht auf der Fraktionsseite verlinkt wurde. Dies ist der Fall bei den folgenden Politikern:

"EstherDilcher", "wiebkeesdar1", "FahimiYasmin", "FechnerJohannes", "sigmargabriel", "GustavHerzogMdB", "AnetteKramme", "LischkaB", "Doro_Martin", "oezdemir_spd", "MechthildRawert", "SusannRuethrich", "rainerspiering", 

Die Tweets aus den Timelines dieser Politiker wurden entsprechend im Juni in den Datensatz aufgenommen.

Für die folgenden Namen gab es Auffälligkeiten, weshalb sie nicht enthalten sind (Grund angegeben):
* GabiHillerOhm existiert nicht (mehr)
* FechnerJohannes - 0 tweets
* kahrs (Johannes Kahrs (36)) hat Stand 10.05. seinen Twitter Account deaktiviert (Mandatsverzicht 06.05.2020)
* Tweets von Martin Rabanus martinrabanus (72) sind geschützt - am 10.05. immernoch pending => aus Liste gelöscht
* BaradariNezahat - 0 tweets
* marcobuelow - aus der Fraktion ausgetreten (27.11.2018)
* BurkertMartin - Mandatsverzicht 01.02.2020
* GerdesMdB - 0 Tweets
* UlrichKelber - private meinung auf twitter
* AndreaNahlesSPD - Mandatsverzicht 31.10.2019
* ManjaSchuele - private meinung auf twitter
* Swen Schulz - Account existiert nicht mehr


In [2]:
import pymongo

client = pymongo.MongoClient("mongodb://127.0.0.1:27017/")

db = client['Twitter']
# All_Tweets_collection = db['twitter_mdp_ex_date_proj'] # Flache Collection mit weniger Feldern
All_Tweets_collection = db['Twitter_mdp_extend']         # Basis in der zunächst alle Tweets gesammelt werden

Über den Zeitraum Feb-Aug 2020 wurde der Datensatz zu verschiedenen Zeitpunkten aktualisiert. Dabei ergaben sich folgende Probleme:
* die Option "since" in $\texttt{.user_timeline}$ wird ignoriert wenn man gleichtzeitig "count" oder "page" verwendet
* beim laden in die modifizierte Collection Twitter_mp_datetime wurde ein duplicate error geworfen
* durch die Arbeit auf zwei Computern muss die Datenbank bei Aktualisierung synchronisiert werden

In [4]:
import json

with open('Namen_SPD.json') as json_file:
    SPD = json.load(json_file)
    
SPD = SPD[86:87]
#print(SPD)

for Abgeordneter in SPD:
    i = 1
    user = Abgeordneter
    print(user)
    all_tweets = []
    while i <= 100 :
        tweets_of_page = api.user_timeline(screen_name = user, tweet_mode = "extended", count = 200, page = i, include_rts = True)
        #print(len(tweets_of_page))
        all_tweets = all_tweets + tweets_of_page
        i=i+1
        if len(tweets_of_page) == 0:
            break    # no more tweets from this user
            
    for tweet in all_tweets:
        All_Tweets_collection.insert_one(tweet._json)
    print(len(all_tweets))

# SiemtjeMoeller - -> Name geändert (alter Account gesperrt)

swenschulz


TweepError: Not authorized.

In [10]:
import json

with open('Namen_SPD.json') as json_file:
# with open('/home/lisa/Darmstadt/Master Arbeit/05_Data/Scrapy/Twitter_user/Twitter_user/spiders/SPD_twitter_clean.json') as json_file:
    SPD = json.load(json_file)
    
SPD = SPD[85]
print(SPD)

MartinSchulz


#### Liste aller SPD Politiker ohne Daten in Twitter_mdp

Für manche Politiker findet man keine Daten in Twitter_mdp, wenn man mit dem Screen_name aus der Liste SPD_twitter_clean sucht.

Grund: Die Usernamen in der Liste von der Fraktionsseite (_twitter_clean) weichen ab vom tatsächlichen Screen_name

-> Twitter API ist NICHT case sensitiv, MongoDB und Python aber schon!

=> Option mit RegEx zu suchen (_twitter_Namen) ODER einen Tweet laden und exakten user_screen_name speichern (siehe unten - _twitter_realNames)

In [1]:
# import json
# with open('/home/lisa/Darmstadt/Master Arbeit/05_Data/Scrapy/Twitter_user/Twitter_user/spiders/SPD_twitter_clean.json') as json_file:
#     SPD = json.load(json_file)

# SPD_Name = SPD
# mdp_not_mongodb = []

# for name, n in zip(SPD, range(0,len(SPD))):
#     count = 0
#     #cursor = All_Tweets_collection.find({"user.screen_name": {'$regex':name,'$options':'i'}})
#     cursor = All_Tweets_collection.find({"user.screen_name": name})
#     for doc in cursor:
#         count=count+1
        
#     if count == 0:
#         print(count)
#         print(name)
#         #mdp_not_mongodb.append(name)
#         SPD_Name[n] = All_Tweets_collection.find_one({"user.screen_name": {'$regex':name,'$options':'i'}})['user']['screen_name']
   
        
# print(mdp_not_mongodb)
# with open("/home/lisa/Darmstadt/Master Arbeit/05_Data/Scrapy/Twitter_user/Twitter_user/spiders/SPD_twitter_Names.json", "a") as write_file:
# #with open("mdp_not_mongodb_SPD.json", "w") as write_file:
#      json.dump(SPD_Name, write_file)



#### Erstellen der Liste mit den tatsächlichen screen_names 

In [3]:
# import json

# with open('/home/lisa/Darmstadt/Master Arbeit/05_Data/Scrapy/Twitter_user/Twitter_user/spiders/SPD_twitter_clean.json') as json_file:
#     SPD= json.load(json_file)
    
# #SPD = SPD[]
# names = []

# for Abgeordneter in Union:
#     user = Abgeordneter
#     print(user)
#     all_tweets = api.user_timeline(screen_name = user, count = 1, include_rts = True)
#     for tweet in all_tweets:
#         name = tweet.user.screen_name
#         names.append(name)
        
# print(names)

# with open ("SPD_twitter_realNames", "w") as write_file:
#     json.dump(names, write_file)
    

mvabercron
peteraltmaier
PeterAumer
DoroBaer
Thomas_Bareiss
MaikBeermann
Manfredbehrens
c_bernstiel
Peter_Beyer
MBiadaczMdB
steffenbilger
HBraun
brehm_innbgnord
HeikeBrehmerMdB
rbrinkhaus
gitta_connemann
Hansjoerg_Durz
TErndl
UweFe
Axel_Fischer
thorsten_frei
HPFriedrichCSU
MichaelFrieser
fuchtel
thomasgebhart
groehe
MGrosseBroemer
GruebelMdb
FGuentzler
olavgutting
ChristianHaase6
hahnflo
juergenhardt
MatthiasHauer
MarkHauptmann
MatthiasHeider
mechthildheil
ThomasHeilmann
frankheinrich
MarcHenrichmann
christianhirte
HHirte
tj_tweets
Ingmar_Jung
AnjaKarliczek
KartesMdB
StefanKaufmann
RonjaKemmer
RKiesewetter
Kiessling_MdB
VolkmarKlein
axel_knoerig
jenskoeppen
KoobMar
guenterkrings
ruedigerkruse
MichaelKuffer
Dr_Roy_Kuehne
UlrichLange
DrLaunert
kleikert
DerLenzMdB
AndreaLindholz
NikolasLoebel
JM_Luczak
DaniLudwigMdB
MaagKarin
YvonneMagwas
ManderlaGisela
AMattfeldt
meister_schafft
janmetzler
hans_michelbach
MdBMonstadt
karstenmoering
ElisabethMotsc1
mueller_sepp
smuellermdb
DrAndreasNick
W